# Pcap analysis

In [162]:
import pandas as pd
from scapy.all import *
from scapy.contrib.coap import *
from scapy.contrib.mqtt import *
from collections import Counter

In [163]:
packets = rdpcap('capture.pcapng')

CQ1) How many different Confirmable PUT requests obtained an 
unsuccessful response from the local CoAP server?

In [164]:
# Select Confirmable PUT request
put_con_packets = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code == 3 and pkt[CoAP].type == 0]

# Select only requests with unsuccessful responce from the local CoAP server
uns_resp = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code > 69 and pkt["IP"].src == "127.0.0.1"]

count = len({pkt[CoAP].token for pkt in put_con_packets} & {pkt[CoAP].token for pkt in uns_resp})

print("Answer:")
print(count)

Answer:
22


CQ2) How many CoAP resources in the coap.me public server received the 
same number of unique Confirmable and Non Confirmable GET requests?

Assuming a resource receives X different CONFIRMABLE requests and Y different 
NONCONFIRMABLE  GET requests, how many resources have X=Y, with X>0?

In [165]:
con_resources = Counter()
non_resources = Counter()

for pkt in packets: 
    if pkt.haslayer(CoAP) and pkt[CoAP].code == 1 and pkt["IP"].dst == "134.102.218.18":
        uri_path_str = ""
        for option in pkt[CoAP].options:
            if option[0] == "Uri-Path":
                uri_path = option[1].decode('utf-8')
                if uri_path:
                    uri_path_str = uri_path_str + "/" + uri_path
        if pkt[CoAP].type == 0:    
            con_resources[uri_path_str]+=1
        else:
            non_resources[uri_path_str]+=1
            

resources = [r for r in (set(con_resources.keys()) & set(non_resources.keys())) if con_resources[r] == non_resources[r]]

print(resources)

print("Answer:")
print(len(resources))

['/validate', '/secret', '/large']
Answer:
3


CQ3) How many different MQTT clients subscribe to the public broker 
HiveMQ using multi-level wildcards?

In [166]:
clients = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 8 and (pkt.dst == "35.158.43.69" or pkt.dst == "35.158.34.213" or pkt.dst == "18.192.151.104"):
        if len(pkt[MQTT].topics) == 1:
            if "#" in pkt[MQTT].topics[0].topic.decode("utf-8") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)
        else: # MQTT 5.0 packet
            if "#" in pkt[MQTT].topics[1].load.decode("utf-8") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)

print(clients)
print("Answer:")
print(len(clients))

[38641, 38619, 54449, 57863]
Answer:
4


CQ4) How many different MQTT clients specify a last Will Message to be 
directed to a topic having as first level "university"?

In [167]:
clients = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 1:
        try:
            if pkt[MQTT].willflag == 1 and pkt[MQTT].willtopic.decode("utf-8").startswith("university/") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)
        except: # MQTT 5.0 packet
            mqtt_payload = pkt[MQTT].load
            if len(mqtt_payload) > 7:  # Il byte dei flag deve esistere
                connect_flags = mqtt_payload[7]  # Il secondo byte del pacchetto è il flag
                will_flag = (connect_flags & 0b00000100) >> 2  # Estrai il 3° bit
                
                if will_flag == 1:

                    index = 8  # Dopo i Connect Flags
                    # Salta il Keep Alive (2 byte)
                    index += 2  
                    # Legge la Property Length (solo in MQTT 5.0)
                    property_length = mqtt_payload[index]  
                    index += 1 + property_length  # Salta anche tutte le proprietà
                    # Legge la lunghezza del Client ID
                    client_id_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2 + client_id_len  # Salta il Client ID
                    # Controlla se ci sono Will Properties (solo in MQTT 5.0)
                    will_property_length = mqtt_payload[index]
                    index += 1 + will_property_length  # Salta le Will Properties
                    # Trova la lunghezza del Will Topic
                    will_topic_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2  # Sposta l'indice all'inizio del Will Topic
                    # Estrai il Will Topic
                    will_topic = mqtt_payload[index:index+will_topic_len].decode()

                    if will_topic.startswith("university/") and pkt[TCP].sport not in clients:
                        clients.append(pkt[TCP].sport)

print(clients)
print("Answer:")
print(len(clients))

[38083]
Answer:
1


CQ5) How many MQTT subscribers receive a last will message derived from 
a subscription without a wildcard?

In [168]:
topics = []
subscribers = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 1:
        try:
            if pkt[MQTT].willflag == 1:
                will_topic = pkt[MQTT].willtopic.decode("utf-8")
        except: # MQTT 5.0 packet
            mqtt_payload = pkt[MQTT].load
            if len(mqtt_payload) > 7:  # Il byte dei flag deve esistere
                connect_flags = mqtt_payload[7]  # Il secondo byte del pacchetto è il flag
                will_flag = (connect_flags & 0b00000100) >> 2  # Estrai il 3° bit
                
                if will_flag == 1:

                    index = 8  # Dopo i Connect Flags
                    # Salta il Keep Alive (2 byte)
                    index += 2  
                    # Legge la Property Length (solo in MQTT 5.0)
                    property_length = mqtt_payload[index]  
                    index += 1 + property_length  # Salta anche tutte le proprietà
                    # Legge la lunghezza del Client ID
                    client_id_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2 + client_id_len  # Salta il Client ID
                    # Controlla se ci sono Will Properties (solo in MQTT 5.0)
                    will_property_length = mqtt_payload[index]
                    index += 1 + will_property_length  # Salta le Will Properties
                    # Trova la lunghezza del Will Topic
                    will_topic_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2  # Sposta l'indice all'inizio del Will Topic
                    # Estrai il Will Topic
                    will_topic = mqtt_payload[index:index+will_topic_len].decode()
                    
        if will_topic not in topics and not any(wildcard in will_topic for wildcard in "+#"):
                    topics.append(will_topic)

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 8:
        if len(pkt[MQTT].topics) == 1:
            if pkt[MQTT].topics[0].topic.decode("utf-8") in topics and pkt[TCP].sport not in subscribers:
                subscribers.append(pkt[TCP].sport)
        else: # MQTT 5.0 packet
            if pkt[MQTT].topics[1].load.decode("utf-8") in topics and pkt[TCP].sport not in subscribers:
                subscribers.append(pkt[TCP].sport)
    
print(topics)
print(subscribers)
print("Answer:")
print(len(subscribers))

['university/department12/room1/temperature', 'metaverse/room2/floor4', 'hospital/facility3/area3', 'metaverse/room2/room2']
[39551, 53557, 41789]
Answer:
3


CQ6) How many MQTT publish messages directed to the public broker 
mosquitto are sent with the retain option and use QoS “At most once”?

In [169]:
count = 0
for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 3 and pkt.dst == "5.196.78.28" and pkt[MQTT].RETAIN == 1 and pkt[MQTT].QOS == 0:
        count += 1

print("Answer:")
print(count)

Answer:
208


CQ7) How many MQTT-SN messages on port 1885 are sent by the clients to 
a broker in the local machine?

In [170]:
count = 0
for pkt in packets:
    if pkt.haslayer(UDP) and pkt[UDP].dport == 1885:
        count += 1

print("Answer:")
print(count)

Answer:
0
